In [1]:
import mlflow
import os
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Step 2: Set up the MLflow tracking server
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))

In [2]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from imblearn.over_sampling import ADASYN

In [3]:
df = pd.read_csv('D:\datascience\campusx\PROJECTS\Second_project\cleaned_data.csv',index_col=0).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [4]:
mlflow.set_experiment("EXP_5-Xgboost_hpt")

<Experiment: artifact_location='s3://apoorv-mlfow-bucket/650043532633511350', creation_time=1758612848676, experiment_id='650043532633511350', last_update_time=1758612848676, lifecycle_stage='active', name='EXP_5-Xgboost_hpt', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [6]:

# Step 1: Remove rows with NaN labels
df = df.dropna(subset=['category'])

X_raw = df['clean_comment']
y_raw = df['category']

# Step 2: Train-test split first
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# Step 3: TF-IDF vectorizer fit on training data only
ngram_range = (1, 3)
max_features = 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# Step 4: Apply ADASYN only on training set
adasyn = ADASYN(random_state=42)
X_train_vec_resampled, y_train_resampled = adasyn.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, best_params=None):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        if best_params:
            for param, value in best_params.items():
                mlflow.log_param(param, value)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Step 5: Optuna objective for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec_resampled, y_train_resampled).predict(X_test_vec))

# Step 6: Run Optuna and log the best model & parameters
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    best_params = study.best_params
    print("Best parameters:", best_params)
    print("Best accuracy:", study.best_value)

    best_model = LogisticRegression(
        C=best_params['C'],
        penalty=best_params['penalty'],
        solver='liblinear',
        random_state=42
    )

    log_mlflow(
        "LogisticRegression", 
        best_model, 
        X_train_vec_resampled, 
        X_test_vec, 
        y_train_resampled, 
        y_test, 
        best_params=best_params
    )

# Run the experiment
run_optuna_experiment()


[I 2025-09-23 22:24:27,159] A new study created in memory with name: no-name-5e014a92-18f4-4d26-a20c-6783a8a506ff
c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-09-23 22:24:27,479] Trial 0 finished with value: 0.45261148234010634 and parameters: {'C': 0.00011246606467235444, 'penalty': 'l2'}. Best is trial 0 with value: 0.45261148234010634.
c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep ap

Best parameters: {'C': 0.5345867446632488, 'penalty': 'l1'}
Best accuracy: 0.7961270966862131


c:\Users\apoor\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
2025/09/23 22:25:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/23 22:25:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression_ADASYN_TFIDF_Trigrams at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350/runs/7751b73b359040349e6bc2521099b9f8
🧪 View experiment at: http://ec2-13-48-43-101.eu-north-1.compute.amazonaws.com:5000/#/experiments/650043532633511350
